In [74]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras_vggface.vggface import VGGFace


In [75]:
#custom parameters
nb_class = 5

vgg_model = VGGFace(model='resnet50',include_top=False, input_shape=(224, 224, 3))

94699520/94694792 [==============================] - 38s 0us/step


In [76]:
#vgg_model.load_weights("C:\\Users\\isaac yang\\.keras\\models\\vggface\\rcmalli_vggface_tf_notop_resnet50.h5")

In [77]:
vgg_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_5[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_197 (Activation)     (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [78]:
vgg_model.trainable = True
vgg_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_5[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_197 (Activation)     (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [79]:
for layer in vgg_model.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True
    else:
        layer.trainable = False


In [80]:
vgg_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_5[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_197 (Activation)     (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

Total params: 23,561,152
Trainable params: 53,120
Non-trainable params: 23,508,032
__________________________________________________________________________________________________


In [81]:
model = Sequential()
model.add(vgg_model)

In [82]:
classifier_model=Sequential()


In [83]:
classifier_model.add(vgg_model)
classifier_model.add(Flatten())


# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('relu'))
# classifier_model.add(Dropout(0.3))
# classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('relu'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(nb_class, activation = 'softmax'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [84]:
model = classifier_model

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_resnet50 (Model)     (None, 1, 1, 2048)        23561152  
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
activation_246 (Activation)  (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                20490     
_________________________________________________________________
batch_normalization_2 (Batch (None, 10)                40        
__________

In [85]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

img_width = img_height = 224
channels = 3
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    featurewise_center=True,
    zca_whitening=True,
    zca_epsilon=1e-06,
    #rotation_range=40, #I'm thinking if a person tilts his head it will be 40 degrees maximum
    #width_shift_range=0.5, #If up to half the person's face is visible width-wise
    #height_shift_range=0.5, #If up to half the person's face is visible height-wise
    #brightness_range=[0.2,2.0], #20% or 200% brightness
    #shear_range=10.0, #putting it very small here because I can't find any tutorials with exact values of good shear range angles
    #zoom_range=[0.3,3], #zooming up to 3 times and minimizing up to 3 times
    #channel_shift_range= 200, #Increases color variation in case someone's face suddenly becomes very tanned, can check the rgb values table
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255,
    validation_split=0.3,
)



In [86]:
class FixedImageDataGenerator(ImageDataGenerator):
    def standardize(self, x):
        if self.featurewise_center:
            x = ((x/255.) - 0.5) * 2.
        return x
datagen = FixedImageDataGenerator(datagen)

In [87]:
batch_size = 16
train_it = datagen.flow_from_directory('data/train2/',
                                       target_size = (img_height, img_width),
                                       class_mode='categorical', 
                                       batch_size=batch_size)

val_it = datagen.flow_from_directory('data/val2/', 
                                    class_mode='categorical',
                                    target_size = (img_height, img_width),
                                    batch_size=batch_size)


Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [88]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
filepath="C:\\development\\NDSC_junior\\SRP\\weights\\resnet50\\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

# earlystop = EarlyStopping(monitor = 'val_loss',
#                          min_delta = 0,
#                          patience = 3,
#                          verbose = 1,
#                          restore_best_weights = True)
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss', 
                             verbose=1,
                             save_best_only=True, 
                             mode='min')

nb_train_samples = 64
nb_validation_samples = 16

epochs = 20
steps_per_epoch = nb_train_samples/batch_size
validation_steps = nb_validation_samples/batch_size

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta = 0.0001, patience=2, min_lr=0, cooldown = 0)
callbacks = [checkpoint, reduce_lr]


model.compile(loss = 'categorical_crossentropy',
             optimizer = 'Adam',
             metrics = ['accuracy'])

In [89]:
model.fit_generator(train_it, 
                    validation_data = val_it, 
                    steps_per_epoch=steps_per_epoch, 
                    validation_steps=validation_steps, 
                    epochs = epochs, 
                    callbacks= callbacks) 

Epoch 1/20
4/4 [==============================] - 86s 22s/step - loss: 1.8777 - acc: 0.1562 - val_loss: 1.5035 - val_acc: 0.1875

Epoch 00001: val_loss improved from inf to 1.50348, saving model to C:\development\NDSC_junior\SRP\weights\resnet50\weights-improvement-01-1.50.hdf5
Epoch 2/20
4/4 [==============================] - 22s 5s/step - loss: 1.5788 - acc: 0.3773 - val_loss: 1.0361 - val_acc: 0.6667

Epoch 00002: val_loss improved from 1.50348 to 1.03615, saving model to C:\development\NDSC_junior\SRP\weights\resnet50\weights-improvement-02-1.04.hdf5
Epoch 3/20
4/4 [==============================] - 23s 6s/step - loss: 1.2670 - acc: 0.3931 - val_loss: 1.0555 - val_acc: 0.5000

Epoch 00003: val_loss did not improve from 1.03615
Epoch 4/20
4/4 [==============================] - 23s 6s/step - loss: 1.0573 - acc: 0.5781 - val_loss: 0.7203 - val_acc: 0.7778

Epoch 00004: val_loss improved from 1.03615 to 0.72034, saving model to C:\development\NDSC_junior\SRP\weights\resnet50\weights-im

KeyboardInterrupt: 